# TODO: TITLE

## Introduction

Many people around the world suffer from heart disease for reasons. Heart problems are common among older adults due to the aging of the body. Heart disease patients often have angina as a symptom. In addition, not only does blood pressure reflect the health of blood vessels, but serum cholesterol values can also indicate potential blood flow restriction in blood vessels.  
- What is the predictive status of vessel stenosis based on patient age, chest pain type, resting blood pressure, and serum cholesterol values?
- Identify/explore dataset

## Preliminary Data Analysis

- Demonstrate dataset can be read into R
- Clean and wrangle data
- Summarize data in at least one table
- Vizualize the data with at least one plot

## Methods
- Explain how we will conduct our data analysis and what variables/columns we will use
    - Age
    - Choloesterol
    - Chest pain type
    - Resting heart rate
- Describe at least one way we will visualize results

## Expected Outcomes
- What do we expect to find?
- What impact could such findings have?
- What future questions could this lead to?